## TSTR(Train on synthetic, test on real)

In [2]:
import numpy as np

In [3]:
REAL_FILE_NAME = 'inputs/sin_wave.npz'  # real data for mmd
GENERETED_FILE_NAME = 'outputs/samples.npz'  # generated samples

In [17]:
npz = np.load(GENERETED_FILE_NAME)
X_train, y_train = npz['x'], npz['y']
print(X_train.shape)
print(y_train.shape)

(2000, 30, 1)
(2000,)


In [18]:
npz = np.load(REAL_FILE_NAME)
X_test, y_test = npz['x'], npz['y']
print(X_test.shape)
print(y_test.shape)

(9000, 30, 1)
(9000,)


In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers import Dropout
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

In [22]:
# hyper parameter for training
seq_length = X_train.shape[1]
input_dim = X_train.shape[2]
num_classes = 3
num_epochs = 10
batch_size = 32
hidden_dim = 128
# to one-hot vector
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [23]:
model = Sequential()
model.add(LSTM(units=hidden_dim,
               batch_input_shape=(None, seq_length, input_dim),
               return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop",
              metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=0, verbose=1)

In [24]:
fit = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=[X_test, y_test],
          callbacks=[early_stopping])

Train on 2000 samples, validate on 9000 samples
Epoch 1/10
2000/2000 [==============================] - 9s 5ms/step - loss: 0.6389 - acc: 0.6525 - val_loss: 0.5408 - val_acc: 0.6569
Epoch 2/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5092 - acc: 0.7210 - val_loss: 0.4100 - val_acc: 0.8557
Epoch 3/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.3785 - acc: 0.8565 - val_loss: 0.2329 - val_acc: 0.9144
Epoch 4/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.2197 - acc: 0.9310 - val_loss: 0.0980 - val_acc: 0.9906
Epoch 5/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1335 - acc: 0.9675 - val_loss: 0.0652 - val_acc: 0.9893
Epoch 6/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0981 - acc: 0.9780 - val_loss: 0.0502 - val_acc: 0.9892
Epoch 7/10
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0788 - acc: 0.9810 - val_loss: 0.0667 - val_acc: 0.9824
Epoch 00007: e